In [1]:
import local_models.local_models
import local_models.algorithms
import local_models.utils
import local_models.linear_projections
import local_models.loggin
import local_models.TLS_models
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
import sklearn.cluster
from importlib import reload
from ml_battery.utils import cmap
import matplotlib as mpl
import sklearn.datasets
import sklearn.decomposition
import logging
import ml_battery.log
import time
import os
import mayavi
import mayavi.mlab
import matplotlib.image as img


import subprocess
#on headless systems, tmux: "Xvfb :1 -screen 0 1280x1024x24 -auth localhost", then "export DISPLAY=:1" in the jupyter tmux
mayavi.mlab.options.offscreen = True



logger = logging.getLogger(__name__)

#reload(local_models.local_models)
#reload(lm)
#reload(local_models.loggin)
#reload(local_models.TLS_models)
np.warnings.filterwarnings('ignore')


In [78]:
def import_shit():
    import local_models.local_models
    import local_models.algorithms
    import local_models.utils
    import local_models.linear_projections
    import local_models.loggin
    import local_models.TLS_models
    import numpy as np
    import logging
    import ml_battery.log

    logger = logging.getLogger(__name__)

    #reload(local_models.local_models)
    #reload(lm)
    #reload(local_models.loggin)
    #reload(local_models.TLS_models)
    np.warnings.filterwarnings('ignore')
    return logger

In [3]:
FRESH=True

In [4]:
kernel_names = {
    local_models.local_models.GaussianKernel: 'gaussian',
    local_models.local_models.TriCubeKernel: 'tricube'
}

In [5]:
mpl.rcParams['figure.figsize'] = [8.0, 8.0]

In [43]:
def img2data(image):
    return np.concatenate((np.mgrid[0:1:image.shape[0]*1j, 0:1:image.shape[1]*1j], image.reshape(1,*image.shape)), axis=0)

In [62]:
import glob
class Everything(object):
    def __eq__(*args): return True

# data is time, y, x, intensity
imgs = []
previous_dims = Everything()
for i, im_path in enumerate(glob.glob("../data/sintel/training/clean/alley_1/*.png")):
    image = np.average(img.imread(im_path), axis=2)
    dims = image.shape
    if not dims==previous_dims:
        raise BaseException("must all be same shape")
    previous_dims = dims
    imgs.append(np.concatenate((np.ones([1] + list(image.shape[:2]))*i, img2data(image))))
data = np.stack(imgs, axis=3)
data[0,:,:,:] /= i
gridder = local_models.utils.Grid2Vec()
data = gridder.fit_transform(data)

In [63]:
data.shape

(22323200, 4)

In [71]:
KERNEL=local_models.local_models.GaussianKernel
RUN = 1

project_dir = "../data/sintel_alley1_{}_{:03d}".format(kernel_names[KERNEL], RUN)
#project_dir = "../data/3dscene_{}".format(kernel_names[KERNEL])

os.makedirs(project_dir, exist_ok=1)



In [65]:
def read_flo(f_name):
    with open(f_name, 'rb') as f:
        magic = np.fromfile(f, np.float32, count=1)
        if 202021.25 != magic:
            raise BaseException("invalid flo file {}".format(f_name))
        else:
            w = np.fromfile(f, np.int32, count=1)[0]
            h = np.fromfile(f, np.int32, count=1)[0]
            data = np.fromfile(f, np.float32, count=2*w*h)
            # Reshape data into 3D array (columns, rows, bands)
            data2D = np.resize(data, (h, w, 2))
    return data2D

def EPE(flo1, flo2):
    return np.average(np.linalg.norm(flo1-flo2, axis=2))
    

In [67]:
linear_models = local_models.local_models.LocalModels(local_models.TLS_models.LinearODR_mD(3))
linear_models.fit(data)

In [72]:
BANDWIDTH = 0.03
pth = os.path.join(project_dir, "bandwidth_{:08.03f}".format(BANDWIDTH))
os.makedirs(pth, exist_ok=1)

In [82]:
def converge_n_save(bandwidth, i, data):
    logger = import_shit()
    logger.info("stuff!")
    for converged in local_models.algorithms.local_tls_shift_till_convergence(linear_models, data, tol=1e-8,
                                         kernel=KERNEL(bandwidth),
                                         report=False):
        pass
    logger.info("batch {:05d} complete")
    np.savetxt(os.path.join(pth, "converged{:08d}.dat".format(i)), converged)
    return None

def batcher(data, size):
    for i in range(0, data.shape[0], size):
        batch = data[i:i+size]
        print(batch.shape)
        yield batch
    
if FRESH:
    from joblib import Parallel, delayed
    convergededs = Parallel(n_jobs=32)(delayed(converge_n_save)(BANDWIDTH, i, batch) for i, batch in enumerate(batcher(data, 100)))
else:
    convergededs = []
    for i, bandwidth in enumerate(bandwidths):
        convergededs.append(np.loadtxt(os.path.join(pth, "converged{:08.02f}.dat".format(bandwidth))))


(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(100, 4)
(

KeyboardInterrupt: 

In [ ]:
def mean_center(data, weights=None):
    return data - np.average(data, axis=0,weights=weights)

In [60]:
for i, bandwidth in enumerate(bandwidths):
    converged = convergededs[i]
    data_avg = np.average(converged, axis=0)
    data_std = np.std(converged, axis=0)
    figure = mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(1000, 500))
    #mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
    nodes = mayavi.mlab.points3d(converged[:,0], converged[:,1], converged[:,2], 
                                 scale_mode='scalar', colormap='gist_earth')
    nodes.glyph.scale_mode = 'scale_by_vector'
    nodes.mlab_source.dataset.point_data.vectors = np.ones((converged.shape[0],3))*(np.average(data_std)/400)
    nodes.mlab_source.dataset.point_data.scalars = (converged[:,1] - (data_avg[1]-3*data_std[1]))/(6*data_std[1])
    #bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
    #bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


    #mayavi.mlab.axes()

    #mayavi.mlab.view(views[0][1],views[0][0])
    mayavi.mlab.view(azimuth=180, elevation=80, distance=1*np.average(data_avg), focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
    mayavi.mlab.move(forward=None, right=0.1*data_avg[1], up=-0.02*data_avg[0])
    title = "converged_data_b{:08.02f}".format(bandwidth)
    mayavi.mlab.savefig(os.path.join(pth, "{}.png".format(title)))
    mayavi.mlab.close(figure)

In [40]:
mayavi.mlab.close(all=True)

In [ ]:
range_pct = 0.2
grid_steps = 20
data_mins, data_maxes, data_ranges = local_models.linear_projections.min_max_range(data)
graph_bounds = local_models.linear_projections.sane_graph_bounds(data_mins, data_maxes, data_ranges, range_pct)

grid_limits = tuple(map(lambda i: slice(graph_bounds[0,i], graph_bounds[1,i], grid_steps*1j), range(graph_bounds.shape[1])))
grid = np.mgrid[grid_limits]
gridder = local_models.utils.Grid2Vec()
grid = gridder.fit_transform(grid)

In [ ]:
for converged2 in local_models.algorithms.local_tls_shift_till_convergence(linear_models, grid, tol=1e-10,
                                     kernel=KERNEL(bandwidth),
                                     report=False):
    pass

In [ ]:
converged2

In [ ]:
cloud = pcl.PointCloud()
cloud.from_array(converged2.astype('float32'))
#visual = pcl.pcl_visualization.CloudViewing()
#visual.ShowMonochromeCloud(cloud)

In [ ]:
mayavi.mlab.figure(figure=None, bgcolor=(1,1,1), fgcolor=(0,0,0), engine=None, size=(800, 800))
#mayavi.mlab.surf(grid[0], grid[1], kde_wireframe/z_scale, colormap='Greys', opacity=1)
nodes = mayavi.mlab.points3d(converged2[:,0], converged2[:,1], converged2[:,2], scale_mode='scalar', color=(1,0,0))
nodes.mlab_source.dataset.point_data.scalars = np.ones(converged2.shape[0])*0.1
bunodes = mayavi.mlab.points3d(data[:,0], data[:,1], data[:,2], scale_mode='scalar', color=(0,1,0))
bunodes.mlab_source.dataset.point_data.scalars = np.ones(data.shape[0])*0.1


#mayavi.mlab.axes()

#mayavi.mlab.view(views[0][1],views[0][0])
data_avg = np.average(converged2, axis=1)
mayavi.mlab.view(azimuth=0, elevation=30, distance=1, focalpoint=(data_avg[0], data_avg[1], data_avg[2]))
title = "converged_g000{:05d}".format(grid_steps)
mayavi.mlab.savefig(os.path.join(project_dir, "{}.png".format(title)))
mayavi.mlab.clf()